# Crime Data Analysis

- Your analysis here
  
---

In [22]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib

# File to Load (Remember to Change These)
data_2010_2019 = Path("Crime_Data_from_2010_to_2019 (1).csv")
data_2020_2023 = Path("Crime_Data_from_2020_to_Present_20231016.csv")

# Read data(2010-2019) and data (2020- present)Data File and store into Pandas DataFrames
pre_covid_data = pd.read_csv(data_2010_2019)
post_covid_data = pd.read_csv(data_2020_2023)
#pre_covid_data.head(5)
#post_covid_data.head(5)

# Combine the data into a single dataset.  
Data_complete = pd.concat([pre_covid_data, post_covid_data], ignore_index=True)
Data_complete.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,AREA
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13.0,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695,NaN
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14.0,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962,NaN
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13.0,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524,NaN
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6.0,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295,NaN
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1.0,Central,176,1,122,"RAPE, ATTEMPTED",...,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488,NaN


In [23]:
Data_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951497 entries, 0 to 2951496
Data columns (total 29 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DR_NO           int64  
 1   Date Rptd       object 
 2   DATE OCC        object 
 3   TIME OCC        int64  
 4   AREA            float64
 5   AREA NAME       object 
 6   Rpt Dist No     int64  
 7   Part 1-2        int64  
 8   Crm Cd          int64  
 9   Crm Cd Desc     object 
 10  Mocodes         object 
 11  Vict Age        int64  
 12  Vict Sex        object 
 13  Vict Descent    object 
 14  Premis Cd       float64
 15  Premis Desc     object 
 16  Weapon Used Cd  float64
 17  Weapon Desc     object 
 18  Status          object 
 19  Status Desc     object 
 20  Crm Cd 1        float64
 21  Crm Cd 2        float64
 22  Crm Cd 3        float64
 23  Crm Cd 4        float64
 24  LOCATION        object 
 25  Cross Street    object 
 26  LAT             float64
 27  LON             float64
 28  AREA        

Data Analysis


In [24]:
len(Data_complete.columns)

29

In [25]:
#Drop the columns not needed
Data_complete.drop(Data_complete.columns[[3, 4, 6, 7, 10, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]], axis=1, inplace=True)

# Now, you can access Data_complete without any issues
Data_complete.head()


,DR_NO,Date Rptd,DATE OCC,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Desc,LAT,LON,AREA
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,Newton,900,VIOLATION OF COURT ORDER,48,M,H,SINGLE FAMILY DWELLING,33.9825,-118.2695,NaN
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,Pacific,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,STREET,33.9599,-118.3962,NaN
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,Newton,946,OTHER MISCELLANEOUS CRIME,0,M,H,ALLEY,34.0224,-118.2524,NaN
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,Hollywood,900,VIOLATION OF COURT ORDER,47,F,W,STREET,34.1016,-118.3295,NaN
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,Central,122,"RAPE, ATTEMPTED",47,F,H,ALLEY,34.0387,-118.2488,NaN


In [ ]:
#Clean Data to include only years only 2016 to 2019
# Clean data for duplicate
#Data_complete = Data_complete.drop_duplicates(subset=['DR_NO'])
# Clean data for -ve values

# Remove enteries with  Victim Sex : X and Victim Race: X, Removing 0 in VictAge

In [ ]:
clean_crime_data.info()

In [ ]:
# Our data should be uniquely identified by DR ID

In [ ]:
#Check all Incident number are unique

In [ ]:
# Change the name of the columns

## Crimes Summary

In [ ]:
# Calculate the total number of Crimes 
Total_crime_count = 


# unique kind of the crimes:

# Pick which highest frequency (5)

# Inside the home and outside the home( CoVID peopel were home)



In [ ]:
#comparing the Total  number of crimes over time



In [ ]:
# Calculate the kind of crimes (e.g different kind of crimes) per year??
#the distribution of crime types over the years
df_2017= 
df_2018
df_2019
df_2020
df_2021
df_2022
crime_type



In [ ]:
# Calculate the average number crime_type from 2016 to 2019 for the 5 crimes we selected


In [ ]:
# Calculate the average number crime_type from 2020 to 2021 for the 5 crimes we selected

In [ ]:
#  Calculate the percentage number crime_type from 2016 to 2019 ?

In [ ]:
# Calculate the percentage number crime_type from 2020 to present ?

In [ ]:
# Create a dataframe with crime_type and year????
crime_summary = pd.DataFrame({
    

In [ ]:
# Plot a bar plot with multiple columns over the different years for total number of crimes(value count)

In [ ]:
# Plot another histogram for average of pre covid and post covid crimes (Total and one for each crime).

## Area Summary

In [ ]:
#  select all of the different Areas


In [ ]:
# Divide areas into Central, Valley, South, West

In [ ]:
# Calculate the total crimes per area per year
per_area_crime_counts = 

In [ ]:
#calculate crime types per area
# Five highest crimes 

In [ ]:
# Geoplot the area 

In [ ]:
# Make a data frame with Columns for Average crime per area


# Display Data Frame

In [ ]:
# Highest Crime Area(By Total Crimes)

In [ ]:
# Lowest Crime Area (By Total Crime)

In [ ]:
#Bar chart four areas, four years and total number of crimes

In [ ]:
# Identify Hot Spots
#Crime in each area acroos the years

In [ ]:
# Any change in the Hot spots overs the years


### Female Vs Male Victims

In [ ]:
# Generate a pie plot showing the distribution of female versus male victims using Pandas
data = data["Sex"].value_counts()
plt.title("Female vs. Male Victims")
Female_male_data.plot.pie(autopct= "%1.1f%%")
plt.show()

### Victim Race 

# Child Abuse compare over the years

## Change in crime spot

In [ ]:
#Splitting race by groups: Black, White, Hispanics, Asians

In [ ]:
#Splitting premise into 4 categories: Commercial, residential, industrial and outdoors
# using Bins

In [ ]:
#Percentage of increase and decrease in crimes over the years
Total_crime_count groupbyyear (count)

In [ ]:
 #How has crime changed over the years?
 
 Line graphs 